# Company revenues data wrangling

In [1]:
import pandas as pd
from scipy import stats
import feather

## Load data

_Filter out companies headquartered in excluded countries and those for which any 2016-2018 revenues are missing_

In [2]:
excluded_countries = ['China', 'Hong Kong', 'Macau', 'Taiwan']

df = pd.read_excel(
        'data/raw/GEOREV CHINA v2.xlsx',
        sheet_name='Tab1',
        header=4,
        usecols=[*range(0, 6), 23, 24, 33, 35],
    ) \
    .rename({
        'GeoRev Ctry Rev -  China 2018': 'china_rev_18',
        'GeoRev Ctry Rev -  China 2017': 'china_rev_17',
        'GeoRev Ctry Rev -  China 2016': 'china_rev_16',
        'GeoRev Country Conf -  China 2018': 'confidence_18',
        'GeoRev Country Conf -  China 2017': 'confidence_17',
        'FactSet Econ Sector': 'fs_sector',
        'Entity Country HQ': 'country_hq'
    }, axis=1) \
    .query(
        'country_hq not in @excluded_countries & \
        china_rev_18 > 0 & \
        china_rev_17 > 0 & \
        china_rev_16 > 0'
    ) \
    .reset_index(drop=True)

print(len(df))

df.head()

3147


,Symbol,Name,Proper Name,china_rev_18,china_rev_17,china_rev_16,confidence_18,confidence_17,fs_sector,country_hq
0,AAPL,Apple Inc.,Apple Inc.,48610.113265,41223.586487,44548.065336,0.994835,0.994016,Electronic Technology,United States
1,CVX,Chevron Corporation,Chevron Corporation,43015.038617,15130.650400,11968.969261,0.984912,0.982324,Energy Minerals,United States
2,677172,"Samsung Electronics Co., Ltd.","Samsung Electronics Co., Ltd.",38128.177592,32830.270880,29672.845543,0.995439,0.995088,Electronic Technology,South Korea
3,B03MM4,Royal Dutch Shell Plc Class B,Royal Dutch Shell Plc Class B,28160.476338,6168.474093,10010.409158,0.995771,0.994704,Energy Minerals,Netherlands
4,B4T3BW,Glencore plc,Glencore plc,27851.338477,23736.513549,3868.434231,0.988919,0.987635,Non-Energy Minerals,Switzerland


In [3]:
df.dtypes

Symbol            object
Name              object
Proper Name       object
china_rev_18     float64
china_rev_17     float64
china_rev_16     float64
confidence_18    float64
confidence_17    float64
fs_sector         object
country_hq        object
dtype: object

In [4]:
df['country_hq'].unique()

array(['United States', 'South Korea', 'Netherlands', 'Switzerland',
       'Germany', 'United Kingdom', 'Australia', 'Singapore',
       'Russian Federation', 'Saudi Arabia', 'France', 'Ireland',
       'Brazil', 'Japan', 'Sweden', 'New Zealand', 'Finland', 'Denmark',
       'Thailand', 'Canada', 'Norway', 'Spain', 'Turkey', 'Malaysia',
       'Austria', 'Italy', 'Israel', 'Cayman Islands', 'Bermuda',
       'South Africa', 'Luxembourg', 'Poland', 'United Arab Emirates',
       'Indonesia', 'Kuwait', 'Chile', 'Qatar', 'Mexico', 'Jordan',
       'Belgium', 'Panama', 'Philippines', 'Argentina', 'Iceland',
       'Egypt', 'Bahrain', 'Cyprus', 'Pakistan', 'India', 'Costa Rica',
       'Vietnam', 'Greece', 'Lithuania', 'Mauritius', 'Kazakhstan',
       'Croatia', 'Oman', 'Portugal', 'Bangladesh', 'Puerto Rico',
       'Macedonia', 'Barbados', 'Colombia', 'Malta', 'Nigeria',
       'Sri Lanka', 'Bosnia and Herzegovina', 'Trinidad and Tobago',
       'Slovenia', 'Gibraltar', 'British Virgin 

## Consolidate and rename sectors per Gabe's feedback

_Filter out finance sector_

In [5]:
sectors = df['fs_sector'].unique() \
    .tolist()

sectors.sort()

print(sectors)

['@NA', 'Commercial Services', 'Communications', 'Consumer Durables', 'Consumer Non-Durables', 'Consumer Services', 'Distribution Services', 'Electronic Technology', 'Energy Minerals', 'Finance', 'Health Services', 'Health Technology', 'Industrial Services', 'Miscellaneous', 'Non-Energy Minerals', 'Process Industries', 'Producer Manufacturing', 'Retail Trade', 'Technology Services', 'Transportation', 'Utilities']


In [6]:
sector_mapping = {
    'Consumer Durables': 'Consumer and autos',
    'Consumer Non-Durables': 'Consumer and autos',
    'Consumer Services': 'Consumer and autos',
    'Process Industries': 'Light manufacturing, processing',
    'Producer Manufacturing': 'Heavy industry, machinery',
    'Electronic Technology': 'Electronic equipment, semiconductors, aerospace, defense',
    'Technology Services': 'IT, software, internet',
    'Utilities': 'Others',
    'Communications': 'Others',
    'Industrial Services': 'Others',
    'Commercial Services': 'Others',
    'Miscellaneous': 'Others',
    'Energy Minerals': 'Energy and mining',
    'Non-Energy Minerals': 'Energy and mining',
    'Transportation': 'Retail, wholesale, logistics, shipping',
    'Distribution Services': 'Retail, wholesale, logistics, shipping',
    'Retail Trade': 'Retail, wholesale, logistics, shipping',
    'Health Services': 'Health services and technology',
    'Health Technology': 'Health services and technology'
}

In [7]:
df_new_sectors = df.copy() \
    .query('fs_sector not in ["Finance", "@NA"]') \
    .assign(curated_sector = lambda x: x['fs_sector'].apply(
        lambda x: sector_mapping[x] if x in sector_mapping.keys() else x
    )) \
    .drop('fs_sector', axis=1) \
    .reset_index(drop=True)

print(len(df_new_sectors))

df_new_sectors.head()

2899


,Symbol,Name,Proper Name,china_rev_18,china_rev_17,china_rev_16,confidence_18,confidence_17,country_hq,curated_sector
0,AAPL,Apple Inc.,Apple Inc.,48610.113265,41223.586487,44548.065336,0.994835,0.994016,United States,"Electronic equipment, semiconductors, aerospac..."
1,CVX,Chevron Corporation,Chevron Corporation,43015.038617,15130.650400,11968.969261,0.984912,0.982324,United States,Energy and mining
2,677172,"Samsung Electronics Co., Ltd.","Samsung Electronics Co., Ltd.",38128.177592,32830.270880,29672.845543,0.995439,0.995088,South Korea,"Electronic equipment, semiconductors, aerospac..."
3,B03MM4,Royal Dutch Shell Plc Class B,Royal Dutch Shell Plc Class B,28160.476338,6168.474093,10010.409158,0.995771,0.994704,Netherlands,Energy and mining
4,B4T3BW,Glencore plc,Glencore plc,27851.338477,23736.513549,3868.434231,0.988919,0.987635,Switzerland,Energy and mining


In [8]:
df_new_sectors['curated_sector'].unique().tolist()

['Electronic equipment, semiconductors, aerospace, defense',
 'Energy and mining',
 'Consumer and autos',
 'Light manufacturing, processing',
 'Retail, wholesale, logistics, shipping',
 'IT, software, internet',
 'Heavy industry, machinery',
 'Health services and technology',
 'Others']

## Check for and drop duplicate companies

In [9]:
duplicate_revs_18 = df_new_sectors['china_rev_18'].duplicated(keep=False)
duplicate_revs_17 = df_new_sectors['china_rev_17'].duplicated(keep=False)
duplicate_revs_16 = df_new_sectors['china_rev_16'].duplicated(keep=False)

df_duplicate_revs = pd.concat([
        df_new_sectors[duplicate_revs_18],
        df_new_sectors[duplicate_revs_17],
        df_new_sectors[duplicate_revs_16]
    ]) \
    .drop_duplicates('Name') \
    .sort_values('Name')

df_duplicate_revs

,Symbol,Name,Proper Name,china_rev_18,china_rev_17,china_rev_16,confidence_18,confidence_17,country_hq,curated_sector
7,614469,BHP Group Ltd,BHP Group Ltd,22273.495007,18270.128419,12748.531058,0.986029,0.984850,Australia,Energy and mining
6,BH0P3Z,BHP Group Plc,BHP Group Plc,22273.495007,18270.128419,12748.531058,0.986029,0.984850,United Kingdom,Energy and mining
306,CCL.U,Carnival Corporation,Carnival Corporation,521.107104,1634.065769,2148.537916,0.996536,0.995428,United States,Consumer and autos
307,312152,Carnival plc,Carnival plc,521.107104,1634.065769,2148.537916,0.996536,0.995428,United Kingdom,Consumer and autos
1117,264551,Gerdau SA Pfd,Gerdau SA Pfd,45.502057,13.939814,33.179163,0.999684,0.999537,Brazil,Energy and mining
247,B0CCH4,Heineken Holding N.V.,Heineken Holding N.V.,737.172169,654.856583,626.028324,0.996523,0.995788,Netherlands,Consumer and autos
259,779255,Heineken NV,Heineken NV,690.016962,654.856583,626.028324,0.996745,0.995791,Netherlands,Consumer and autos
1004,LTRPA,Liberty TripAdvisor Holdings Inc Class A,Liberty TripAdvisor Holdings Inc Class A,59.482340,95.853222,94.989485,0.995663,0.990791,United States,Consumer and autos
1599,264886,Metalurgica Gerdau SA Pfd,Metalurgica Gerdau SA Pfd,12.262780,13.939814,33.179163,0.999684,0.999538,Brazil,Energy and mining
711,*QSR,Restaurant Brands International Inc,Restaurant Brands International Inc,126.355934,40.457529,33.490767,0.997173,0.996279,Canada,Consumer and autos


In [10]:
df_deduped = df_new_sectors.copy() \
    .drop([
#         BHP: drop UK (they're HQed in Australia)
        6,
#         Carnival: drop UK (they're HQed in US)
        307,
#         Heineken: drop holding company
        247,
#         TripAdvisor: drop holding company
        1004,
#         Gerdau: same revenues except 2018; drop one with lower 2018 revenue
        1599,
#         Restaurant Brands International: drop limited partnership
        712,
#         Rio Tinto: drop UK (they're HQed in Australia)
        10,
#         Unilever: drop Netherlands (same revenues, doesn't matter)
        23
    ]) \
    .reset_index(drop=True)

print(len(df_deduped))

df_deduped.head()

2891


,Symbol,Name,Proper Name,china_rev_18,china_rev_17,china_rev_16,confidence_18,confidence_17,country_hq,curated_sector
0,AAPL,Apple Inc.,Apple Inc.,48610.113265,41223.586487,44548.065336,0.994835,0.994016,United States,"Electronic equipment, semiconductors, aerospac..."
1,CVX,Chevron Corporation,Chevron Corporation,43015.038617,15130.650400,11968.969261,0.984912,0.982324,United States,Energy and mining
2,677172,"Samsung Electronics Co., Ltd.","Samsung Electronics Co., Ltd.",38128.177592,32830.270880,29672.845543,0.995439,0.995088,South Korea,"Electronic equipment, semiconductors, aerospac..."
3,B03MM4,Royal Dutch Shell Plc Class B,Royal Dutch Shell Plc Class B,28160.476338,6168.474093,10010.409158,0.995771,0.994704,Netherlands,Energy and mining
4,B4T3BW,Glencore plc,Glencore plc,27851.338477,23736.513549,3868.434231,0.988919,0.987635,Switzerland,Energy and mining


## Calculate overall revenue growth

In [11]:
revenue_18 = df_deduped['china_rev_18'].sum()
revenue_17 = df_deduped['china_rev_17'].sum()
revenue_16 = df_deduped['china_rev_16'].sum()
growth_18 = (revenue_18 - revenue_17) / revenue_17 * 100
growth_17 = (revenue_17 - revenue_16) / revenue_16 * 100

print(f'2018 growth: {round(growth_18, 1)} per cent')
print(f'2017 growth: {round(growth_17, 1)} per cent')

2018 growth: 19.8 per cent
2017 growth: 13.9 per cent


## Calculate individual companies’ revenue growth

In [12]:
df_cos_growth = df_deduped.copy() \
    .assign(growth_18 = lambda x:
        round((x['china_rev_18'] - x['china_rev_17']) / x['china_rev_17'], 6) * 100
    ) \
    .assign(growth_17 = lambda x:
        round((x['china_rev_17'] - x['china_rev_16']) / x['china_rev_16'], 6) * 100
    ) \
    .assign(faster_18 = lambda x: x['growth_18'] > x['growth_17'])

print(len(df_cos_growth))

df_cos_growth.head()

2891


,Symbol,Name,Proper Name,china_rev_18,china_rev_17,china_rev_16,confidence_18,confidence_17,country_hq,curated_sector,growth_18,growth_17,faster_18
0,AAPL,Apple Inc.,Apple Inc.,48610.113265,41223.586487,44548.065336,0.994835,0.994016,United States,"Electronic equipment, semiconductors, aerospac...",17.9182,-7.4627,True
1,CVX,Chevron Corporation,Chevron Corporation,43015.038617,15130.650400,11968.969261,0.984912,0.982324,United States,Energy and mining,184.2907,26.4157,True
2,677172,"Samsung Electronics Co., Ltd.","Samsung Electronics Co., Ltd.",38128.177592,32830.270880,29672.845543,0.995439,0.995088,South Korea,"Electronic equipment, semiconductors, aerospac...",16.1373,10.6408,True
3,B03MM4,Royal Dutch Shell Plc Class B,Royal Dutch Shell Plc Class B,28160.476338,6168.474093,10010.409158,0.995771,0.994704,Netherlands,Energy and mining,356.5226,-38.3794,True
4,B4T3BW,Glencore plc,Glencore plc,27851.338477,23736.513549,3868.434231,0.988919,0.987635,Switzerland,Energy and mining,17.3354,513.5949,False


In [13]:
total_cos = len(df_cos_growth)
faster_18_cos = len(df_cos_growth.query('faster_18 == True'))
faster_18_pc = round(faster_18_cos / total_cos, 2) * 100

print(f'''Within this group, {faster_18_pc} per cent of companies
posted faster growth rates in 2018 compared with
2017, counterbalancing the deceleration or
outright revenue decline of the rest.''')

Within this group, 54.0 per cent of companies
posted faster growth rates in 2018 compared with
2017, counterbalancing the deceleration or
outright revenue decline of the rest.


## Assign Z-scores and percentiles

In [14]:
df_z = df_cos_growth.copy() \
    .assign(growth_18_z = lambda x: stats.zscore(x['growth_18'])) \
    .assign(growth_18_p = lambda x: pd.qcut(x['growth_18'], 100, labels=[*range(1, 101)])) \
    .sort_values('growth_18', ascending=False) \
    .reset_index(drop=True)

print(len(df_z))

df_z.head()

2891


,Symbol,Name,Proper Name,china_rev_18,china_rev_17,china_rev_16,confidence_18,confidence_17,country_hq,curated_sector,growth_18,growth_17,faster_18,growth_18_z,growth_18_p
0,BLDY62,Rhipe Limited,Rhipe Limited,15.458523,0.095392,0.480210,0.992843,0.999522,Australia,"Heavy industry, machinery",16105.1969,-80.1353,True,28.634923,100
1,661448,Webzen Inc.,Webzen Inc.,45.966671,0.328330,0.202569,0.993882,0.999543,South Korea,"IT, software, internet",13900.1495,62.0827,True,24.692860,100
2,B03MXR,Turkiye Sise ve Cam Fabrikalari A.S.,Turkiye Sise ve Cam Fabrikalari A.S.,90.761591,0.702733,0.657526,0.996689,0.999964,Turkey,"Light manufacturing, processing",12815.5167,6.8753,True,22.753813,100
3,BF1GQB,P.C.S. Machine Group Holding Public Company Ltd,P.C.S. Machine Group Holding Public Company Ltd,22.611381,0.323594,0.091828,0.980272,0.999418,Thailand,"Heavy industry, machinery",6887.5744,252.3908,True,12.156162,100
4,VBIV,"VBI Vaccines, Inc.","VBI Vaccines, Inc.",2.590077,0.043343,0.043849,0.979546,0.994639,United States,Health services and technology,5875.7547,-1.1541,True,10.347287,100


In [15]:
df_z.tail()

,Symbol,Name,Proper Name,china_rev_18,china_rev_17,china_rev_16,confidence_18,confidence_17,country_hq,curated_sector,growth_18,growth_17,faster_18,growth_18_z,growth_18_p
2886,B014JZ,Thaicom Public Co. Ltd.,Thaicom Public Co. Ltd.,0.034680,6.187502,8.195417,1.000000,0.999005,Thailand,Others,-99.4395,-24.5005,False,-0.334838,1
2887,*HEMP,Hempco Food and Fiber Inc.,Hempco Food and Fiber Inc.,0.002542,0.529729,0.316009,0.999811,0.984474,Canada,Consumer and autos,-99.5202,67.6309,False,-0.334983,1
2888,IDCC,"InterDigital, Inc.","InterDigital, Inc.",0.300088,74.616655,149.734537,1.000000,0.995553,United States,"Electronic equipment, semiconductors, aerospac...",-99.5978,-50.1674,False,-0.335121,1
2889,HL,Hecla Mining Company,Hecla Mining Company,0.064094,67.405020,34.072914,1.000000,0.996332,United States,Energy and mining,-99.9049,97.8258,False,-0.335670,1
2890,659266,Sinwa Ltd.,Sinwa Ltd.,0.000860,1.813345,0.763432,0.997030,0.997030,Singapore,"Retail, wholesale, logistics, shipping",-99.9526,137.5254,False,-0.335756,1


In [16]:
feather.write_dataframe(df_z, 'data/processed/company-revenues.feather')

## Calculate sectoral revenue and growth

In [17]:
df_sectors = df_deduped.copy() \
    .groupby('curated_sector', as_index=False)[
        'china_rev_18',
        'china_rev_17',
        'china_rev_16',
        'confidence_18',
        'confidence_17'
    ] \
    .agg({
        'china_rev_18': 'sum',
        'china_rev_17': 'sum',
        'china_rev_16': 'sum',
        'confidence_18': 'mean',
        'confidence_17': 'mean'
    
    }) \
    .assign(sector_growth_18 = lambda x:
        round((x['china_rev_18'] - x['china_rev_17']) / x['china_rev_17'], 6) * 100
    ) \
    .assign(sector_growth_17 = lambda x:
        round((x['china_rev_17'] - x['china_rev_16']) / x['china_rev_16'], 6) * 100
    ) \
    .assign(slower_18 = lambda x: x['sector_growth_18'] < x['sector_growth_17']) \
    .drop('china_rev_16', axis=1) \
    .sort_values('sector_growth_18', ascending=False) \
    .reset_index(drop=True)


print(len(df_sectors))

df_sectors

9


,curated_sector,china_rev_18,china_rev_17,confidence_18,confidence_17,sector_growth_18,sector_growth_17,slower_18
0,Energy and mining,242016.741407,163185.375811,0.995139,0.994462,48.3079,31.4231,False
1,"IT, software, internet",47642.046425,37938.649907,0.995452,0.994811,25.5765,15.6723,False
2,Health services and technology,59225.919681,48955.624952,0.995464,0.994873,20.9788,11.4335,False
3,"Electronic equipment, semiconductors, aerospac...",274578.018589,233920.840281,0.994018,0.993183,17.3807,8.7726,False
4,"Heavy industry, machinery",123882.054476,106652.015991,0.995311,0.994859,16.1554,9.7131,False
5,"Light manufacturing, processing",102447.873262,88986.807666,0.994462,0.993809,15.1270,13.4077,False
6,Others,40823.692907,37357.881197,0.996099,0.995603,9.2773,31.9221,True
7,Consumer and autos,170877.860873,157625.254803,0.995335,0.994918,8.4077,9.8821,True
8,"Retail, wholesale, logistics, shipping",69476.606155,69695.619982,0.995452,0.995277,-0.3142,6.1380,True


In [18]:
feather.write_dataframe(df_sectors, 'data/processed/sector-revenues.feather')